In [1]:
from Bio import SeqIO
import pandas as pd
import re
from pyfaidx import Fasta
from tqdm import tqdm

In [2]:
# Define the function that will find all 'CG'
def findCpGs(chrom, genome, breed):
    seq = str(genome[chrom].seq)
    #comp_seq = str(genome[chrom].seq.complement())
    cpg_coords = open(f'./{breed}_{chrom}.CpGs.bed', 'w')
    #chrom = chrom.split('_')[0]
    #cpg_comp_coords = open(f'./{chrom}.complement.CpGs.bed', 'w')

    for match in re.finditer(r'cg', seq, re.IGNORECASE):
        cpg_coords.write(f'{chrom}\t{match.start()}\t{match.start() + 2}\t.\t.\t+\n')
    #    cpg_comp_coords.write(f'{chrom}\t{match.start() + 1}\t{match.start() + 2}\t.\t.\t-\n')
    #for match in re.finditer(r'gc', comp_seq, re.IGNORECASE):
    #    cpg_comp_coords.write(f'{chrom}\t{match.start() + 1}\t{match.start() + 2}\t.\t.\t-\n')

    cpg_coords.close()
    #cpg_comp_coords.close()

    return

In [ ]:
seq = [f'{str(i)}' for i in range(1,30)]

path = 'path/to/your/reference_genome'

genome = SeqIO.to_dict(
    SeqIO.parse(path,'fasta'))

for i in tqdm(seq):
    findCpGs(chrom=i, genome = genome, breed = 'Brahman')

cpg_window = 1002

In [ ]:

path = 'path/to/your/reference_genome'
for i in tqdm(range(1,30)):
    cpg_fa = open(f'./Brahman_CpG/Brahman.CpGs.chr{i}.1kb.fa', 'w') # Where you want to save the new 1002bp CpG "reads".
    window = int(cpg_window / 2)
    #fasta = Fasta(path + 'Angus/Angus.ARS-UCD1.2.orientation.fa')
    fasta = Fasta(path + 'Brahman_oriented2_ARS/Bos_indicus_hybrid.UOA_Brahman_1.ARS-UCD1.2.orientation.dna.toplevel.fa')
    cpgs = pd.read_csv(f'./Brahman_CpG/beds/Brahman_{i}.CpGs.bed',
                   header=None,
                   index_col=None,
                   sep='\t',
                   dtype={0:str,
                          1:int,
                          2:int,
                          3:str,
                          4:str,
                          5:str})

    # get 500 bp either side of the CpG.
    for row in cpgs.itertuples():
        #print(row)
        if row[2] - window < 0:
            continue
            #start = 0
            #stop = cpg_window + 2
        elif row[3] + window > len(fasta[f'{i}']):
            continue
            #start = len(fasta[f'{i}']) - (cpg_window + 2)
            #stop = len(fasta[f'{i}'])
        else:
            start = row[2] - window
            stop = row[3] + window
        seq = fasta[str(row[1])][start:stop]
        assert len(seq) == int(cpg_window + 2), f'The length of the sequence is {len(seq)}\n{seq}'
        cpg_fa.write(f'>{row[1]}:{row[2]}-{row[3]} new_CpG_coordinate:{row[1]}:{start}-{stop}'
              f'\n{fasta[str(row[1])][start:stop]}\n')
    cpg_fa.close()